In [8]:

# !conda create --name vulscriber-cluster-env python=3.8
# !conda activate vulscriber-cluster-env
# !conda install pytorch==1.10.0 torchvision==0.11.1 torchaudio==0.10.0 cudatoolkit=11.3 -c pytorch
# !conda install transformers==4.37.2 pandas scikit-learn


In [9]:
# !pip install accelerate>=0.26.0

In [10]:
# import torch
# print(torch.cuda.is_available())  # Should print: True
# print(torch.cuda.device_count())  # Should print: 1
# print(torch.cuda.get_device_name(0))  # Should print: NVIDIA GeForce RTX 3060
# print(torch.cuda.current_device())  # Should print 0


In [ ]:

from chat import PromptUtils

extract_code_content = PromptUtils.extract_code_content

from chat import CodeQwenPrompter as code_qwen
from chat import PromptUtils as pu

ch = code_qwen(lock=None) # Selects CodeQwen1.5--7B-Chat


c:\ProgramData\miniconda3\envs\vulscriber-cluster-env\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards:  25%|██▌       | 1/4 [00:35<01:47, 35.80s/it]

# Load the LLM and the bigvul's train dataframe

In [ ]:
# Load file JSON và chuyển thành DataFrame
import pandas as pd
import json

MEGAVUL_CLEAN_TRAIN = r"C:\Users\Administrator\Downloads\VulScribeR-main\code\dataset\megavul\new\megavul_clean.json"


# Load file JSON và chuyển thành DataFrame
with open(MEGAVUL_CLEAN_TRAIN, "r", encoding="utf-8") as f:
    clean_json_data = json.load(f)
dftr = pd.DataFrame(clean_json_data)



JSONDecodeError: Extra data: line 2 column 1 (char 1813)

## Introduciton of RAG

In [ ]:
from rag import read_naive_full_header_rag, read_naive_full_bigvul_header_rag, read_naive_full_func_rag, read_naive_code2code_rag, read_naive_code2code_ext_rag, read_random, read_naive_code2code_clustered_rag, read_random_fair

search_results = read_naive_code2code_clustered_rag() # For Injection and Extension Methods - RQ1 and RQ3


In [ ]:
search_results[0].max_score, search_results[1].max_score, search_results[2].max_score, search_results[10].max_score, search_results[100].max_score 
len(search_results),search_results[0]

(7059,
 SearchItem(searchItemId='src/C/cholmod.c', scoredCodeSnippets=[
   ScoredCodeSnippet(id='mono/metadata/icall.c', body='static gboolean
 ves_icall_System_Array_FastCopy (MonoArray *...', header='static gboolean ves_icall_System_Array_FastCopy (MonoArray* source, int source_idx, MonoArray* dest, int dest_idx, int length)', cluster='0', score=4.092301)
   ScoredCodeSnippet(id='codestream/acsequentialscan.hpp', body='void Init(bool hi) 
       {
         for(int i = 0;i < 18;i++...', header='void Init(bool hi)', cluster='1', score=3.7259467)
   ScoredCodeSnippet(id='src/C/cholmod.c', body='static PyObject* spsolve(PyObject *self, PyObject *args,
    ...', header='static PyObject* spsolve(PyObject* self, PyObject* args, PyObject* kwrd)', cluster='2', score=18.262472)
   ScoredCodeSnippet(id='src/imagew-bmp.c', body='static int find_high_bit(unsigned int x)
 {
 	int i;
 	for(i=31...', header='static int find_high_bit(unsigned int x)', cluster='3', score=4.4920135)
   ScoredCodeSnippe

Vì MegaVul không có trường metadata riêng cho vulnerable_lines, chúng ta sẽ giả định rằng các dòng code bị xóa (deleted_lines) trong quá trình sửa lỗi (tức là trong diff_line_info) chính là các dòng chứa hoặc liên quan trực tiếp đến lỗ hổng trong phiên bản code trước đó (func_before).

In [ ]:
def get_vulnerable_lines_from_diff(diff_info):
    """Extracts deleted lines from diff_info as vulnerable lines."""
    if not diff_info or "deleted_lines" not in diff_info:
        return []
    return diff_info["deleted_lines"]

In [ ]:
from rag import flatten_search_items
from operator import attrgetter

def sort(search_results):
    return sorted(search_results, key=attrgetter('max_score'), reverse=True)

print(len(search_results)) 
search_results = flatten_search_items(search_results) # flattens search results in case the format is (1 clean) -> 5 vuls (for RQ2) instead of 1-1 matching, otherwise it doesn't change anything

# Extracts deleted lines from diff_info, assuming they are the vulnerable lines.
# for result in search_results:
#     print(result)
#     # diff_info = result.get("diff_line_info", {})
#     diff_info = getattr(result, "diff_line_info", {})
#     print(diff_info)
#     result["vulnerable_lines"] = get_vulnerable_lines_from_diff(diff_info)  # Store extracted lines

search_results = sort(search_results) # sort the results (if clustering is used, this will be invalidated and the clustered sampling algorithm's sorting method will be used)
print(len(search_results))


7059
35167


In [ ]:

avg = 0
max_avg = 0
avg_m = 0
for r in search_results:
    if avg_m < r.avg_score:
        avg_m = r.avg_score
    avg += r.avg_score / len(search_results)
    max_avg += r.max_score/len(search_results)

print(avg, max_avg, avg_m)

10.350285964307089 10.350285964307089 89.021286


In [ ]:
def clustered_sampling(search_results):
    if search_results[0].scoredCodeSnippets[0].cluster == -1:
        print("Search results are for a non-clustered strategy, skipping clustred_sampling")
        return search_results

    cluster_order = [4,3,1,0,2] # sorted based on size
    cluster_indices = [0,0,0,0,0]

    new_search_results = []

    search_results_per_cluster = {0:[], 1:[], 2:[],3:[],4:[]}

    for sr in search_results:
        search_results_per_cluster[sr.scoredCodeSnippets[0].cluster].append(sr) # putting items of each cluster to it's array

    for i in range(5):
        search_results_per_cluster[i] = sort(search_results_per_cluster[i]) # sort each cluster's results

    for i in range(len(search_results)):
        cluster_number = cluster_order[i%5] # sampling from sorted clusters while starting from the largest cluster
        cluster_index = cluster_indices[cluster_number]

        if cluster_index < len(search_results_per_cluster[cluster_number]):
            new_search_results.append(search_results_per_cluster[cluster_number][cluster_index])
            cluster_indices[cluster_number] += 1
    
    print("Clusted Sampling was used...")
    return new_search_results

search_results = clustered_sampling(search_results)

Clusted Sampling was used...


In [ ]:
search_results[0].max_score, search_results[1].max_score, search_results[2].max_score, search_results[10].max_score, search_results[100].max_score 

(79.94688, 41.326096, 66.86675, 74.269684, 56.64057)

In [ ]:
# resuming_data_frame = pd.read_json("./generated/"somesetting".jsonl", orient="records", lines=True) # just in case more was needed, but didn't want to start from 0 again

In [ ]:
import numpy as np
from rag import SearchItem
from chat import ConcurrencyUtils as cu

def get_index(search_item: SearchItem):
    return str(search_item.best)


def get_vul(search_item: SearchItem):
    best_target_index = search_item.best
    vul_record = dftr[dftr['file_path'] == best_target_index]
    return str(vul_record['func_before'].values[0])

def get_fixed_vul(search_item: SearchItem):
    best_target_index = search_item.best
    vul_record = dftr[dftr['file_path'] == best_target_index]
    return str(vul_record['func'].values[0])

# def get_vul_lines(search_item: SearchItem):
#     best_target_index = search_item.best
#     vul_record = dftr[dftr['file_path'] == best_target_index]
#     flaw_lines = str(vul_record['diff_line_info'].values[0]).strip()
#     if len(flaw_lines) < 5:
#         raise Exception(f"Invalid Vul_lines: {flaw_lines}")
#     return flaw_lines
def get_vul_lines(search_item: SearchItem):
    best_target_index = search_item.best
    vul_record = dftr[dftr['file_path'] == best_target_index]

    # Debugging step: Print matched rows
    print(f"Matching records for {best_target_index}:")
    print(vul_record)

    # Check if record exists
    if vul_record.empty:
        raise Exception(f"No matching record found for file_path: {best_target_index}")

    # Check if `diff_line_info` exists
    if "diff_line_info" not in vul_record.columns:
        raise Exception(f"Missing `diff_line_info` column in dftr!")

    # Extract flaw lines
    flaw_lines = vul_record['diff_line_info'].values[0]

    # Handle missing or empty flaw_lines
    if flaw_lines is None or str(flaw_lines).strip() == "":
        raise Exception(f"Missing or empty `diff_line_info` for: {best_target_index}")

    return str(flaw_lines).strip()



# def get_clean_bigvul(search_item: SearchItem):
#     best_target_index = int(search_item.best)
#     return str(dftr[dftr['index'] == best_target_index]['processed_func'].values[0])

# def get_clean_ext(search_item: SearchItem):
#     retrieved = None
#     try:
#         retrieved = get_clean_megavul(search_item)
#     except Exception as e:
#         # print(f">>getting clean item, devign wasn't found {e}")
#         retrieved = get_clean_bigvul(search_item)
#         # print("<<retrieved from bigvul")
#         assert retrieved != None and len(retrieved) > 0
#     finally:
#         return retrieved

# def get_clean_megavul(search_item: SearchItem):
#     with open(f"./../devign_code/{search_item.searchItemId}", 'r') as file:
#         clean_item = file.read()
#     return clean_item


def get_clean_megavul(search_item: SearchItem):
    # Search for the function by matching `searchItemId` with the `file_path` key
    for item in clean_json_data:
        if item.get("file_path") == search_item.searchItemId:
            return item.get("func", None)  # Return function content if found

    return None  # Return None if not found

# Formulator Functions that take a search result and populates a prompt template

def get_injection_strategy_devign(search_item: SearchItem): # the Injection strategy while the clean's come from devign
    clean_item = get_clean_megavul(search_item)
    vul_code = get_vul(search_item)
    flaw_lines = get_vul_lines(search_item)
    return pu.get_full_prompt(vul_code, clean_item, flaw_lines)


def get_extension_strategy_devign(search_item: SearchItem):
    clean_item = get_clean_megavul(search_item)
    vul_code = get_vul(search_item)
    flaw_lines = get_vul_lines(search_item)
    return pu.get_ext_wl_prompt(clean_item, vul_code, flaw_lines)

def get_mutation_strategy(search_item: SearchItem):
    vul_code = get_vul(search_item)
    flaw_lines = get_vul_lines(search_item)
    return pu.get_mutation_prompt(vul_code, flaw_lines)


# Generator Phase (can send concurrent requests)
indices, vuls, cleans, vul_lines, generated = cu.concurrent_prompter(
    prompt_creator_from_searchItem=get_injection_strategy_devign,
    index_retreiver=get_index,
    vul_retriever=get_vul,
    clean_retriever=get_clean_megavul,
    vul_lines_retriever=get_vul_lines,
    search_results=search_results,
    llm=ch, target=6000, max_workers=8, is_all=False, resume_from_dataframe=None)  # for code Qwen set workers to 1 as there's only one model


NameError: name 'search_results' is not defined

In [ ]:
# import json
# import pandas as pd

# # Đường dẫn file JSON gốc (cập nhật lại đường dẫn theo hệ thống của bạn)
# input_file = r"C:\Users\Administrator\Downloads\VulScribeR-main\dataset\megavul_simple.json"

# # Load file JSON và chuyển thành DataFrame
# with open(input_file, "r", encoding="utf-8") as f:
#     json_data = json.load(f)
# mega_vul_df = pd.DataFrame(json_data)

# # Tách dữ liệu dựa trên trường 'is_vul'
# vul_df = mega_vul_df[mega_vul_df['is_vul'] == True]
# clean_df = mega_vul_df[mega_vul_df['is_vul'] == False]

# # Đường dẫn cho file output (cập nhật lại đường dẫn nếu cần)
# vul_output_file = r"C:\Users\Administrator\Downloads\VulScribeR-main\code\dataset\megavul\new\megavul_vul.json"
# clean_output_file = r"C:\Users\Administrator\Downloads\VulScribeR-main\code\dataset\megavul\new\megavul_clean.json"

# # Ghi kết quả ra file JSON với định dạng 'records'
# vul_df.to_json(vul_output_file, orient="records", force_ascii=False, lines=True)
# clean_df.to_json(clean_output_file, orient="records", force_ascii=False, lines=True)

# print("Tách file thành công!")


Tách file thành công!


In [ ]:
# save the results of the Generator
df_result = pd.DataFrame({'vul': vuls, 'vul_lines': vul_lines, 'clean': cleans, 'generated': generated, 'idx':indices})
df_result.to_json(f"./generated/6k-devign-extension-randomfair.jsonl", orient="records", lines=True)

In [ ]:
# unused strategies

# def get_injection_bigvul(search_item: SearchItem):
#     clean_item = get_clean_bigvul(search_item)
#     vul_code = get_vul(search_item)
#     flaw_lines = get_vul_lines(search_item)
#     return pu.get_full_prompt(vul_code, clean_item, flaw_lines)

# def get_injection_ext(search_item: SearchItem):
#     clean_item = get_clean_ext(search_item)
#     vul_code = get_vul(search_item)
#     flaw_lines = get_vul_lines(search_item)
#     return pu.get_full_prompt(vul_code, clean_item, flaw_lines)
# failed strategies (or the ones that didn't do any better)
# def get_prompt4_devign(search_item: SearchItem):
#     clean_item = get_clean_megavul(search_item)
#     vul_code = get_vul(search_item)
#     flaw_lines = get_vul_lines(search_item)
#     return pu.get_full_prompt_without_examples_and_vardef_guide(vul_code, clean_item, flaw_lines)

# def get_prompt3_devign(search_item: SearchItem):
#     clean_item = get_clean_megavul(search_item)
#     vul_code = get_vul(search_item)
#     flaw_lines = get_vul_lines(search_item)
#     fixed = get_fixed_vul(search_item)
#     return pu.get_full_prompt_with_examples_and_vardef_guide(fixed, vul_code, clean_item, flaw_lines)

# def get_prompt5_devign(search_item: SearchItem):
#     clean_item = get_clean_megavul(search_item)
#     vul_code = get_vul(search_item)
#     return pu.get_ext_prompt(clean_item, vul_code)
